**DEB**

**ANALYSE DU PROBLEME**
---

Problème de classification (28 classes) avec évaluation "sur mesure" (composite) qui tient compte de la performance et de l'équité du modèle<br><br>. 

Jeu de donnée de 25.000 personnes, après voir lancé la baseline, j'ai entrainé un **adversarial NN**, qui permettait de gagner un peu d'équité (passage de 80% à 85% sur le TRP gap) mais a entrainé un éffondrement de l'accuracy (de 65% à 35%). après avoir cherché à personaliser la fonction de perte (avec difficulté pour entrainer le modele), je suis repassée à une approche plus progressive, sur la régression logistique

**regression logisitique (baseline) => 3 problemes**<br>
1. ***l'échantillonage*** : Le "train_test_split" ne tient pas compte de S, nous ne savons pas si l'attribut protégé (H/F) est bien réparti entre X_train et X_test, cela n'est pas optimum pour l'apprentissage car certaines classes contiennent peu de points. Il peut aussi etre plus pertinent de fixer les hyperparametre grace à X_train, et d'entrainer le modele sur tout l'échantillon (avec les memes hyperparametres) avant de prédire X_test_true et de soumettre<br>
=> nous réaliserons des train_test_split en tenant compte de X et Y grace à Y56 = Y + 28*S <br>
=> nous ferons des soumissions double (modele appris sur X_train et aussi X) pour le datachallenge, pour améliorer les qualités prédicitive du modèles <br><br>
2. la fonction d'évaluation est une ***moyenne de scores calculé par classes***, chaque classe a donc le meme poids dans le score final. Or, l'apprentissage est fait pour optimser l'accuracy de l'échantillon (chaque élément à le meme poids). Comme la distribution de l'échatillon est très mal équilibrée entre les classes (93 à 8.285 personnes), le modèle ne peut apprendre correctement<br>
=> nous essaierons pour la forme del a data augmentation, mais sans conviction compte tenu de l'embedding sémantique fait avec BERT<br>
=> nous utiliserons une correction de la fonction de pertes pour y inclure le poids de chaque classe (Y et Y56)<br>
=> nous essaierons du contrastive learning pour obtenir un espace de réprésentation adéquat<br><br>
2. ***l'apprentissage n'est pas optimal (alignement et hyperparametres à améliorer)***. Il manque la régularization et du cross validation. L'apprentissage ,'est pas aligné avec "score final": le modele fait pour optimiser l'accuracy (la précision), or la fonction d'évaluation est une moyenne entre le F1 score (moyenne harmonique de la précision et du rappel) et une métrique de fairness. Le F1 score pénalise les écarts importants entre la précision et le rappel. Le score<br>
=> nous adapterons le fonctions de pertes et/ou l'architecture des modèles intégrer au mieux notrescore final
=> nous ajouterons systematiquement des termes de régularisations et de cross validation (stratified k-fold cross validation si possibl)<br> 

**METHODOLOGIE**
---

**Nous allons procéder en 4 temps** :

**1/ optimiser la régression logisitique**<br>
<br>
ajouter double prédiction sur S et nonS<br><br>

**2/ débiaiser l'échantillon**<br> 


**3/ personaliser la foncition de perts**

**4/ explorer modeles précis à la frontiere**<br>
KNN
SVM linéaire et gaussien
1 versus all sur classe mal prédites

**5/ regression linéaire des modèles**
échantilloner X par rapport à Y et S** traiter le probleme 1 et 2, en créant un label à 56 classes (2 x 28) pour distinguer chaque label i en fonction de s'il est protégé ou non. Nous augmenterons la taille de l'échantillon (avec 56 labels) pour créer X_XL et Y_XL pour avoir le meme nombre d'items pour chacune des 56 classes et un jeu avec le meme nombre d'instance par classe, permettant d'aligner les scores moyens des classes avec la moyenne du nouveau sample "XL" décuplé (passage de 27.749 lignes à 255.304 lignes ). Nous programmerons la régression logisitique avec pytorch pour accélérer les calculs.

2/ nous **ajusterons ensuite la fonction de pertes**, pour qu'elle tienne compte au mieux de la fonction d'évaluation, qui est composée de 2 éléments : le F1 score et le 1-equal ooprtunity gap<br>
- le 'macro f score' (F1 score) n'est pas dérivable (à valider), nous l'approcherons donc par accuracy et le recall<br>
- true positive gap : 1 - TRP_GAP est calculé comme 
A noter que le F1 score n'est pas dérivable. However, we can approximate the F1 score in a differentiable manner by using the ***Sørensen–Dice coefficient***, which is closely related to the F1 score and is differentiable. The Sørensen–Dice coefficient is defined as \(2 * \frac{precision * recall}{precision + recall}\), which is equivalent to the F1 score formula. For a differentiable approximation, we can use soft versions of precision and recall.
<br><br>

3/ Nous explorerons **différents modèles de classification** K-means, SVM, random forest tree<br><br>

4/ Avant de mener cette approche j'avais essayer 



**A) Import, loading data, eval functions**
---

In [1]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import normalize
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.datasets import make_classification
from sklearn.metrics import make_scorer #, f1_score

#from Data_challenge_fairness_2024.evaluator import *
from evaluator import *


In [ ]:

path_model = 'FINAL_models/'  # must finish by '/'
path_Y_pred_true = 'FINAL_Y_pred_true/'  # must finish by '/'

# with open(path_model + name + '.pkl', 'wb') as f: pickle.dump(model, f)
# with open(path_model + name + '.pkl', 'rb') as f: model = pickle.load(f)

def get_final_score(Y_pred, Y, S):
    eval_scores, confusion_matrices_eval = gap_eval_scores(Y_pred, Y, S, metrics=['TPR'])
    final_score = (eval_scores['macro_fscore']+ (1-eval_scores['TPR_GAP']))/2
    return final_score
    
    
def get_scores(Y_pred, Y, S):
    accuracy= accuracy_score(Y, Y_pred)  # Y_test are your original test labels
    print(f"Accuracy on transformed test data: {accuracy}")
    eval_scores, confusion_matrices_eval = gap_eval_scores(Y_pred, Y, S, metrics=['TPR'])
    final_score = (eval_scores['macro_fscore']+ (1-eval_scores['TPR_GAP']))/2
    macro_f1 = eval_scores['macro_fscore']
    inv_macro_gap = 1-eval_scores['TPR_GAP']

    #print results
    print('final score :',final_score)
    print('macro_f1    :',eval_scores['macro_fscore'])
    print('inv_macro_gap',1-eval_scores['TPR_GAP'])
    
    return accuracy, final_score, macro_f1, inv_macro_gap

In [2]:
#####################################################
# Load pickle file and convert to numpy array
#####################################################

with open('data-challenge-student.pickle', 'rb') as handle:
    # dat = pickle.load(handle)
    dat = pd.read_pickle(handle)

#Check keys()
print(dat.keys())
X = dat['X_train']
Y = dat['Y']
S = dat['S_train']

X_test_true = dat['X_test']
S_test_true = dat['S_test']

Y56= Y + 28*S
#X, X_test,Y,S, S_test = dat[1]

print(X.shape,Y.shape,S.shape,X_test_true.shape,S_test_true.shape)

path_model = 'models/'

dict_keys(['X_train', 'X_test', 'Y', 'S_train', 'S_test'])
(27749, 768) (27749,) (27749,) (11893, 768) (11893,)


**LOGISTIC REGRESSION MODELS**
---
<br>
0_Baseline : logistic regression (on X_train)<br><br>
1_Optimised : optimized logistic regression (on X + hyperparameters+ stratified k-fold cross validation)<br><br>
2_Stratified K fold Logistic Regression
3_Orthogonal debiasing of X on model 2 (Optimised Logisitic regression with custom loss function) <br><br>
4_Orthogonal debiasing of X on MAN/WOMAN embedding difference  on model 2 (Optimised Logisitic regression with custom loss function) <br><br>
5_Custom loss fonctin on Optimised Logisitic regression (56 classes with Y56= Y + 28*S) 

In [ ]:
RESULTS = pd.DataFrame(columns=['Model','Accuracy','final score','Fscore macro','1 - TPR gap'])
RESULTS.loc[0]=['baseline (Reglog28)',accuracy_0,final_score_0, eval_scores_0['macro_fscore'],1-eval_scores_0['TPR_GAP']]
RESULTS.loc[2]=['RegLog28 + YxS',accuracy_1,final_score_1, eval_scores_1['macro_fscore'],1-eval_scores_1['TPR_GAP']]
RESULTS.loc[4]=['RegLog28 + XL',accuracy_2,final_score_2, eval_scores_2['macro_fscore'],1-eval_scores_2['TPR_GAP']]
RESULTS.loc[5]=['RegLog28 + YxS + XL',accuracy_3,final_score_3, eval_scores_3['macro_fscore'],1-eval_scores_3['TPR_GAP']]
RESULTS.loc[6]=['RegLog56 + YxS + XL',accuracy_4,final_score_4, eval_scores_4['macro_fscore'],1-eval_scores_4['TPR_GAP']]
RESULTS.loc[3]=['RegLog56 + YxS',accuracy_5,final_score_5, eval_scores_5['macro_fscore'],1-eval_scores_5['TPR_GAP']]
RESULTS.loc[1]=['baseline seed=1 (Reglog28)',accuracy_0_1,final_score_0_1, eval_scores_0_1['macro_fscore'],1-eval_scores_0_1['TPR_GAP']]
RESULTS = RESULTS.sort_index()
RESULTS

with open('RESULTS_10-03-2024.pkl', 'wb') as f:
   pickle.dump(RESULTS, f)

#RESULTS =  pd.read_pickle('RESULTS_10-03-2024.pkl')

**LOGISTIC REGRESSION**
---

In [4]:
###############################################
# 0. LOGISTIC REGRESSION - BASELINE 
###############################################

name ='0_Reglog_baseline' # changer clf_i

# Refresh training data
X_train, X_test, Y_train, Y_test, S_train, S_test = train_test_split(X, Y, S, test_size=0.3, random_state=42)

# training (or load)logistic model
#clf_0 = LogisticRegression(random_state=0, max_iter=5000,verbose=1).fit(X_train, Y_train)
with open(path_model + name + '.pkl', 'rb') as f: clf_0 = pickle.load(f)
model = clf_0 
  
# predicting and assessing
Y_pred = model.predict(X_test)
accuracy, final_score, macro_f1, inv_macro_gap = get_scores(Y_pred,Y_test,S_test)

# predict X_test_true and save
Y_pred_true = model.predict(X_test_true)
results=pd.DataFrame(Y_pred_true, columns= ['score'])
results.to_csv(path_Y_pred_true + "Data_Challenge_" + name + ".csv", header = None, index = None)

# save model
with open(path_model + name + '.pkl', 'wb') as f: pickle.dump(model, f)

Accuracy on transformed test data: 0.7638438438438439
final score : 0.7365033273594812
macro_f1    : 0.669337208333607
inv_macro_gap 0.8036694463853555


In [5]:
##################################################
# 1. LOGISTIC REGRESSION - BASELINE OPTIMISED
# (no validation set - training on all X)
##################################################

name ='1_Reglog_optimised' # changer clf_i

# Refresh training data
# TRAINED ON ALL X

# training (or load)logistic model
# added regularisation 'l2' with coeff 0.2 (C) and early stopping with tol = 0.0001
# clf_1 = LogisticRegression(random_state=42, max_iter=5000,verbose=1,penalty='l2', C=0.2, tol=0.0001).fit(X, Y)
with open(path_model + name + '.pkl', 'rb') as f: clf_1 = pickle.load(f)
model = clf_1

# predicting and assessing
Y_pred = model.predict(X_test)
accuracy, final_score, macro_f1, inv_macro_gap = get_scores(Y_pred,Y_test,S_test)

# predict X_test_true and save
Y_pred_true = model.predict(X_test_true)
results=pd.DataFrame(Y_pred_true, columns= ['score'])
results.to_csv(path_Y_pred_true + "Data_Challenge_" + name + ".csv", header = None, index = None)

# save model
with open(path_model + name + '.pkl', 'wb') as f: pickle.dump(model, f)

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =        21532     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  9.24653D+04    |proj g|=  4.11128D+04


 This problem is unconstrained.



At iterate   50    f=  2.05318D+04    |proj g|=  8.07961D+02

At iterate  100    f=  1.73620D+04    |proj g|=  6.71832D+02

At iterate  150    f=  1.61758D+04    |proj g|=  1.03107D+02

At iterate  200    f=  1.57705D+04    |proj g|=  1.24845D+02

At iterate  250    f=  1.56370D+04    |proj g|=  3.30421D+01

At iterate  300    f=  1.55878D+04    |proj g|=  2.25705D+01

At iterate  350    f=  1.55716D+04    |proj g|=  3.01638D+01

At iterate  400    f=  1.55661D+04    |proj g|=  1.65593D+01

At iterate  450    f=  1.55641D+04    |proj g|=  1.12923D+01

At iterate  500    f=  1.55632D+04    |proj g|=  3.90256D+00

At iterate  550    f=  1.55628D+04    |proj g|=  6.05409D+00

At iterate  600    f=  1.55624D+04    |proj g|=  4.51413D+00

At iterate  650    f=  1.55619D+04    |proj g|=  1.26648D+01

At iterate  700    f=  1.55610D+04    |proj g|=  7.07335D+00

At iterate  750    f=  1.55595D+04    |proj g|=  1.62580D+01

At iterate  800    f=  1.55577D+04    |proj g|=  4.04886D+00

At iter

In [23]:
##################################################
# 2. LOGISTIC REGRESSION - STRATIFIED K-FOLD
# on optimized Log. Reg. regularisatio, early stoppping
# (no validation set - training on all X)
##################################################

name ='2_Reglog_StratKFold' # changer clf_i

from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold, cross_val_score

# Refresh training data
# TRAINED ON ALL X

# training (or load)logistic model
# added regularisation 'l2' with coeff 0.2 (C) and early stopping with tol = 0.0001
clf_2 = LogisticRegression(random_state=42, max_iter=5000,verbose=0,penalty='l2', C=0.2, tol=0.0001)

# Préparer la validation croisée k-fold stratifiée
cv = StratifiedKFold(n_splits=10)
scores = cross_val_score(clf_2, X, Y, scoring='accuracy', cv=cv, n_jobs=-1)
clf_2.fit(X, Y)
model = clf_2
# with open(path_model + name + '.pkl', 'rb') as f: clf_1 = pickle.load(f)

# predicting and assessing
Y_pred = model.predict(X_test)
accuracy, final_score, macro_f1, inv_macro_gap = get_scores(Y_pred,Y_test,S_test)

# predict X_test_true and save
Y_pred_true = model.predict(X_test_true)
results=pd.DataFrame(Y_pred_true, columns= ['score'])
results.to_csv(path_Y_pred_true + "Data_Challenge_" + name + ".csv", header = None, index = None)

# save model
with open(path_model + name + '.pkl', 'wb') as f: pickle.dump(model, f)


Accuracy on transformed test data: 0.856096096096096
final score : 0.8207123104282983
macro_f1    : 0.8407599159632768
inv_macro_gap 0.8006647048933198


In [28]:
##########################################################################
# 3. DEBIASING WITH ORTHOGONAL PROJECTION + OPYIMISED MODEL 
# (using the vector of the average difference of embedding between groups 
# to debiase representation of X, by operating an orthogonal projection)
##########################################################################

name ='3_ReglogState0_Optimized_orthogonal' # changer clf_i

#------------------------------------------------------------------------
# CREATION OF ORTHOGONAL PROJECTION 
# -----------------------------------------------------------------------

# Comparison of average embeddingof sensitive group and non-sensitive group

#calculate average embedding for group sensitive (S=1) and non sensitive (S=0)
X_sensitive = X[S==1]
X_sensitive_mean=X_sensitive.mean()

X_non_sensitive = X[S!=1]
X_non_sensitive_mean=X_non_sensitive.mean()

bias = X_sensitive_mean-X_non_sensitive_mean

# function to debiase data
def remove_bias(dataset, bias):
    # Compute the dot product of each row of the dataset with diff
    dot_products = np.dot(dataset, bias)
    
    # Compute the magnitude of bias
    bias_magnitude_squared = np.dot(bias, bias)
    
    # Compute the projection of each row onto diff
    projection = np.outer(dot_products / bias_magnitude_squared, bias)
    
    # Subtract the projection from the dataset
    debiased_dataset = dataset - projection
    
    return debiased_dataset

#---------------------------------------------------------------------
# Debiasing X (Projection of X orthogonally to bias) 
#---------------------------------------------------------------------

# projection in new representation space
debiased_X = remove_bias(X, bias)

#---------------------------------------------------------------------
# TRAINING MODEL AND PREDICTING
#---------------------------------------------------------------------

clf_3 = LogisticRegression(random_state=0, max_iter=5000,verbose=0,penalty='l2', C=0.2, tol=0.0001).fit(debiased_X, Y)
model = clf_3
# with open(path_model + name + '.pkl', 'rb') as f: clf_3 = pickle.load(f)

# predicting and assessing
Y_pred = model.predict(X_test)
accuracy, final_score, macro_f1, inv_macro_gap = get_scores(Y_pred,Y_test,S_test)

# predict X_test_true and save
modified_X_test_true = remove_info(X_test_true, bias)  # debiasing
Y_pred_true = model.predict(modified_X_test_true)
results=pd.DataFrame(Y_pred_true, columns= ['score'])
results.to_csv(path_Y_pred_true + "Data_Challenge_" + name + ".csv", header = None, index = None)

# save model
with open(path_model + name + '.pkl', 'wb') as f: pickle.dump(model, f)

Accuracy on transformed test data: 0.8545345345345345
final score : 0.8290222283482765
macro_f1    : 0.8381637333117026
inv_macro_gap 0.8198807233848505


In [30]:
##########################################################################
# 3. DEBIASING WITH ORTHOGONAL PROJECTION + MODEL  (stratified K-fold) 
# (using the vector of the average difference of embedding between groups 
# to debiase representation of X, by operating an orthogonal projection)
##########################################################################

name ='3_Reglogstate0_StratKFold_orthogonal' # changer clf_i

#------------------------------------------------------------------------
# CREATION OF ORTHOGONAL PROJECTION 
# -----------------------------------------------------------------------

# Comparison of average embeddingof sensitive group and non-sensitive group

#calculate average embedding for group sensitive (S=1) and non sensitive (S=0)
X_sensitive = X[S==1]
X_sensitive_mean=X_sensitive.mean()

X_non_sensitive = X[S!=1]
X_non_sensitive_mean=X_non_sensitive.mean()

bias = X_sensitive_mean-X_non_sensitive_mean

# function to debiase data
def remove_bias(dataset, bias):
    # Compute the dot product of each row of the dataset with diff
    dot_products = np.dot(dataset, bias)
    
    # Compute the magnitude of bias
    bias_magnitude_squared = np.dot(bias, bias)
    
    # Compute the projection of each row onto diff
    projection = np.outer(dot_products / bias_magnitude_squared, bias)
    
    # Subtract the projection from the dataset
    debiased_dataset = dataset - projection
    
    return debiased_dataset

#---------------------------------------------------------------------
# Debiasing X (Projection of X orthogonally to bias) 
#---------------------------------------------------------------------

# projection in new representation space
debiased_X = remove_bias(X, bias)

#---------------------------------------------------------------------
# TRAINING MODEL AND PREDICTING
#---------------------------------------------------------------------

clf_3 = LogisticRegression(random_state=0, max_iter=5000,verbose=0,penalty='l2', C=0.2, tol=0.0001)

# Préparer la procédure de validation croisée k-fold stratifiée
cv = StratifiedKFold(n_splits=10)
scores = cross_val_score(clf_3, debiased_X, Y,  cv=cv, n_jobs=-1)
clf_3.fit(debiased_X, Y)
model = clf_3
# with open(path_model + name + '.pkl', 'rb') as f: clf_3 = pickle.load(f)

# predicting and assessing
Y_pred = model.predict(X_test)
accuracy, final_score, macro_f1, inv_macro_gap = get_scores(Y_pred,Y_test,S_test)

# predict X_test_true and save
modified_X_test_true = remove_info(X_test_true, bias)  # debiasing
Y_pred_true = model.predict(modified_X_test_true)
results=pd.DataFrame(Y_pred_true, columns= ['score'])
results.to_csv(path_Y_pred_true + "Data_Challenge_" + name + ".csv", header = None, index = None)

# save model
with open(path_model + name + '.pkl', 'wb') as f: pickle.dump(model, f)

In [ ]:
with open(path_model + '1_LogReg_optimised.pkl', 'rb') as f: recup = pickle.load(f)
modified_X_test_true = remove_info(X_test_true, bias)  # debiasing
Y_pred_true_recup = model.predict(modified_X_test_true)
results_recup=pd.DataFrame(Y_pred_true_recup, columns= ['score'])
results_recup.to_csv(path_Y_pred_true + "Data_Challenge_recpu.csv", header = None, index = None)


In [ ]:
'''##########################################################################
# 4. BERT (WOMAN-MAN) DEBIASING WITH ORTHOGONAL PROJECTION + MODEL 2 (k-fold)
# (using the vector of the difference of embedding between WOMAN and MAN
# to debiase representation of X, by operating an orthogonal projection)
# !!!! uncertainty about model of BERT used for the training !!!!
##########################################################################

name ='4_Reglog_StratKFold_orthogonalBERT' # changer clf_i

#------------------------------------------------------------------------
# GET MAN AND WOMAN EMBEDDING IN BERT
#-----------------------------------------------------------------------

from transformers import BertTokenizer, TFBertModel
import tensorflow as tf
import tokenizers as tk
from tqdm import tqdm

# Charger le tokenizer et modèle de BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = TFBertModel.from_pretrained('bert-base-multilingual-cased')

# Tokenize "man" and "woman"
tokens_man = tokenizer.encode("man", add_special_tokens=True)  # Ajoute les tokens spéciaux
tokens_woman = tokenizer.encode("woman", add_special_tokens=True)

print(tokens_man)
print(tokens_woman)

# transforme en tensor de batch (meme si un seul mot)

input_ids_man = tf.constant(tokens_man)[None, :]  # Ajoute une dimension de batch
input_ids_woman = tf.constant(tokens_woman)[None, :]  # Ajoute une dimension de batch

# Obtenir les embeddings
outputs_man = model(input_ids_man)
outputs_woman = model(input_ids_woman)

# Les embeddings du dernier layer pour le premier token ('[CLS]' par défaut)
embedding_man = outputs_man.last_hidden_state[0][0]
embedding_woman = outputs_woman.last_hidden_state[0][0]
bias_BERT = embedding_woman - embedding_man
bias_BERT = np.array (bias_BERT)
print(embedding_man.shape)  # Doit être (768,)
print(embedding_woman.shape)  # Doit être (768,)
print(bias_BERT.shape)  # Doit être (768,)

#---------------------------------------------------------------------
# Debiasing X (Projection of X orthogonally to bias) 
#---------------------------------------------------------------------

# projection in new representation space
debiased_X_BERT = remove_bias(X, bias_BERT)

#----------------------------------------------------------------------
# TRAINING MODEL AND PREDICTING
#------------------------------------------------------------------------

clf_4 = LogisticRegression(random_state=42, max_iter=5000,verbose=1,penalty='l2', C=0.2, tol=0.0001)

# Préparer la procédure de validation croisée k-fold stratifiée
cv = StratifiedKFold(n_splits=10)
#custom_scorer = make_scorer(final_score_S)
scores = cross_val_score(clf_4, debiased_X_BERT, Y, cv=cv, n_jobs=-1)
clf_4.fit(debiased_X_BERT, Y)
model = clf_4
# with open(path_model + name + '.pkl', 'rb') as f: clf_3 = pickle.load(f)

# predicting and assessing
Y_pred = model.predict(X_test)
accuracy, final_score, macro_f1, inv_macro_gap = get_scores(Y_pred,Y_test,S_test)

# predict X_test_true and save
modified_X_test_true = remove_info(X_test_true, bias_BERT)  # debiasing
Y_pred_true = model.predict(modified_X_test_true)
results=pd.DataFrame(Y_pred_true, columns= ['score'])
results.to_csv(path_Y_pred_true + "Data_Challenge_" + name + ".csv", header = None, index = None)

# save model
with open(path_model + name + '.pkl', 'wb') as f: pickle.dump(model, f)'''

In [2]:
''' PROBLEME SUR FINAL SCORE QUI DEPEND DE S, OR SICKIT LEARN NE GERE
PAS S DANS LA CROSS VALIDATION, LA LOSS FUNCTION PERSONALISEE NE PEUT 
ETRE CALCULEE  =======> CODAGE SUR PYTORCH

##########################################################################
# 5. CUSTOM LOSS FUNCTION (Using stratified K + custom_scorer)
# + Debiasing with orthogonal projection + Optimised Logisitic Regression
##########################################################################

name ='5_Reglog_StratKFold1_Orthogonal_CustomLoss' # changer clf_i

#---------------------------------------------------------------------
# Debiasing X (Projection of X orthogonally to bias) 
#---------------------------------------------------------------------

# projection in new representation space
debiased_X = remove_bias(X, bias)

#------------------------------------------------------------------------
# CREATION OF CUSTOM LOSS FUNCTION (FINAL SCORE) 
# -----------------------------------------------------------------------

#It is not possible to split S with the stratified k-fold implementation 
# in sickit learn. We will use Y56 = Y + 28*S to train logistic model
# and derive from Y56 the value of Y = Y56 % 28 and S = Y56//28

def final_score_S(Y,Y_pred):
    # wrapper function to include S
    # Note order : custom scorer expects (y_true, y_pred) as inputs
    
    return get_final_score(Y_pred,Y,S)

custom_scorer = make_scorer(final_score_S)

#---------------------------------------------------------------------
# TRAINING MODEL AND PREDICTING
#---------------------------------------------------------------------

clf_5 = LogisticRegression(random_state=0, max_iter=5000,verbose=0,penalty='l2', C=0.2, tol=0.0001)

# Préparer la procédure de validation croisée k-fold stratifiée
cv = StratifiedKFold(n_splits=1)
scores = cross_val_score(clf_3, debiased_X, Y, scoring=custom_scorer ,cv=cv, n_jobs=-1)
clf_5.fit(debiased_X, Y56)
model = clf_5
# with open(path_model + name + '.pkl', 'rb') as f: clf_5 = pickle.load(f)

# predicting and assessing
Y56_pred = model.predict(X_test)
#S_pred = Y56//28   # facteur (original S) ex 33//28 = 1 (attribut protégé)
Y_pred = Y56_pred % 28  # reste (original Y)   ex 33% 28 = classe 5
accuracy, final_score, macro_f1, inv_macro_gap = get_scores(Y_pred,Y_test,S_test)

# predict X_test_true and save
modified_X_test_true = remove_info(X_test_true, bias)  # debiasing
Y56_pred_true = model.predict(modified_X_test_true)
Y_pred_true = Y56_pred_true % 28  # reste (original Y)   ex 33% 28 = classe 5
results=pd.DataFrame(Y_pred_true, columns= ['score'])
results.to_csv(path_Y_pred_true + "Data_Challenge_" + name + ".csv", header = None, index = None)

# save model
with open(path_model + name + '.pkl', 'wb') as f: pickle.dump(model, f)'''

' PROBLEME SUR FINAL SCORE QUI DEPEND DE S, OR SICKIT LEARN NE GERE\nPAS S DANS LA CROSS VALIDATION, LA LOSS FUNCTION PERSONALISEE NE PEUT \nETRE CALCULEE  =======> CODAGE SUR PYTORCH\n\n##########################################################################\n# 5. CUSTOM LOSS FUNCTION (Using stratified K + custom_scorer)\n# + Debiasing with orthogonal projection + Optimised Logisitic Regression\n##########################################################################\n\nname =\'5_Reglog_StratKFold1_Orthogonal_CustomLoss\' # changer clf_i\n\n#---------------------------------------------------------------------\n# Debiasing X (Projection of X orthogonally to bias) \n#---------------------------------------------------------------------\n\n# projection in new representation space\ndebiased_X = remove_bias(X, bias)\n\n#------------------------------------------------------------------------\n# CREATION OF CUSTOM LOSS FUNCTION (FINAL SCORE) \n# ----------------------------------------

In [25]:
##########################################################################
# 4. CUSTOM LOSS FUNCTION WITH 56 CLASS CLASSIFIER (USING Y56 = Y + S*28)
# + Debiasing with orthogonal projection + Optimised Logisitic Regression
##########################################################################

name ='4_Reglog56_StratKFold_Orthogonal_CustomLoss' # changer clf_i

#---------------------------------------------------------------------
# Debiasing X (Projection of X orthogonally to bias) 
#---------------------------------------------------------------------

# projection in new representation space
debiased_X = remove_bias(X, bias)

#------------------------------------------------------------------------
# CREATION OF CUSTOM LOSS FUNCTION (FINAL SCORE) 
# -----------------------------------------------------------------------

#It is not possible to split S with the stratified k-fold implementation 
# in sickit learn. We will use Y56 = Y + 28*S to train logistic model
# and derive from Y56 the value of Y = Y56 % 28 and S = Y56//28

def final_score_S(Y56,Y_pred):
    '''custom scorer expects (y_true, y_pred) as inputs
    Inputs : Y56 and Y pred
    Outputs : final score on 28 classes with Y56 trick'''
    S = Y56//28   # facteur (original S) ex 33//28 = 1 (attribut protégé)
    Y = Y56 % 28  # reste (original Y)   ex 33% 28 = classe 5 
    return get_final_score(Y_pred,Y,S)

custom_scorer = make_scorer(final_score_S)

#---------------------------------------------------------------------
# TRAINING MODEL AND PREDICTING
#---------------------------------------------------------------------

clf_4 = LogisticRegression(random_state=42, max_iter=5000,verbose=0,penalty='l2', C=0.2, tol=0.0001)

# Préparer la procédure de validation croisée k-fold stratifiée
cv = StratifiedKFold(n_splits=10)
scores = cross_val_score(clf_4, debiased_X, Y56, scoring=custom_scorer ,cv=cv, n_jobs=-1)
clf_4.fit(debiased_X, Y56)
model = clf_4
# with open(path_model + name + '.pkl', 'rb') as f: clf_5 = pickle.load(f)

# predicting and assessing
Y56_pred = model.predict(X_test)
#S_pred = Y56//28   # facteur (original S) ex 33//28 = 1 (attribut protégé)
Y_pred = Y56_pred % 28  # reste (original Y)   ex 33% 28 = classe 5
accuracy, final_score, macro_f1, inv_macro_gap = get_scores(Y_pred,Y_test,S_test)

# predict X_test_true and save
modified_X_test_true = remove_info(X_test_true, bias)  # debiasing
Y56_pred_true = model.predict(modified_X_test_true)
Y_pred_true = Y56_pred_true % 28  # reste (original Y)   ex 33% 28 = classe 5
results=pd.DataFrame(Y_pred_true, columns= ['score'])
results.to_csv(path_Y_pred_true + "Data_Challenge_" + name + ".csv", header = None, index = None)

# save model
with open(path_model + name + '.pkl', 'wb') as f: pickle.dump(model, f)

/home/anaele/myenv/lib/python3.10/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=10.
  warnings.warn(


RUNNING THE L-BFGS-B CODE
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =        43064     M =           10

At X0         0 variables are exactly at the bounds

           * * *

Machine precision = 2.220D-16
 N =        43064     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.00529D+05    |proj g|=  2.06522D+04

At iterate    0    f=  1.00529D+05    |proj g|=  2.06381D+04
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =        43064     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.00529D+05    |proj g|=  2.06118D+04
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =        43064     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.00529D+05    |proj g|=  2.06269D+04


 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =        43064     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.00529D+05    |proj g|=  2.06161D+04
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =        43064     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.00529D+05    |proj g|=  2.06534D+04
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =        43064     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.00529D+05    |proj g|=  2.06381D+04


 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.


...


At iterate   50    f=  2.96015D+04    |proj g|=  4.23754D+02

At iterate   50    f=  3.00049D+04    |proj g|=  3.68947D+03

At iterate   50    f=  2.98276D+04    |proj g|=  3.31223D+02

At iterate   50    f=  2.93594D+04    |proj g|=  1.22474D+03

At iterate   50    f=  2.94112D+04    |proj g|=  5.68829D+02

At iterate   50    f=  2.94696D+04    |proj g|=  1.12163D+03

At iterate   50    f=  3.03922D+04    |proj g|=  6.12752D+02

At iterate   50    f=  2.97034D+04    |proj g|=  2.36665D+03

At iterate  100    f=  2.33063D+04    |proj g|=  2.02884D+03

At iterate  100    f=  2.34883D+04    |proj g|=  3.96808D+02

At iterate  100    f=  2.33991D+04    |proj g|=  7.55888D+02

At iterate  100    f=  2.31575D+04    |proj g|=  2.04184D+02

At iterate  100    f=  2.31886D+04    |proj g|=  5.45021D+02

At iterate  100    f=  2.33333D+04    |proj g|=  1.20937D+03

At iterate  100    f=  2.31890D+04    |proj g|=  1.10662D+03

At iterate  100    f=  2.32847D+04    |proj g|=  2.13262D+02

At iter

 This problem is unconstrained.



At iterate 1700    f=  2.03024D+04    |proj g|=  1.53727D+00

At iterate 1700    f=  2.02723D+04    |proj g|=  5.79839D+00

At iterate 1700    f=  2.03240D+04    |proj g|=  1.02865D+00

At iterate 1700    f=  2.03402D+04    |proj g|=  6.44789D-01

At iterate 1750    f=  2.02466D+04    |proj g|=  1.05729D+00

At iterate 1750    f=  2.02943D+04    |proj g|=  1.11329D+00

At iterate   50    f=  2.97012D+04    |proj g|=  3.05223D+02

At iterate 1700    f=  2.03210D+04    |proj g|=  1.75409D+00

At iterate 1750    f=  2.03240D+04    |proj g|=  4.07429D-01

At iterate 1750    f=  2.03023D+04    |proj g|=  1.77250D+00

At iterate 1800    f=  2.02465D+04    |proj g|=  1.35379D+00

At iterate 1750    f=  2.02723D+04    |proj g|=  1.33830D+00

At iterate 1750    f=  2.03402D+04    |proj g|=  7.74652D-01

At iterate  100    f=  2.33094D+04    |proj g|=  2.01986D+03

At iterate 1800    f=  2.02943D+04    |proj g|=  9.80940D-01

At iterate 1750    f=  2.03210D+04    |proj g|=  9.75117D-01

At iter

 This problem is unconstrained.



At iterate 1950    f=  2.03239D+04    |proj g|=  4.41164D+00

At iterate 2000    f=  2.02463D+04    |proj g|=  2.18766D+00

At iterate 1950    f=  2.03398D+04    |proj g|=  1.96170D+00

At iterate 1950    f=  2.02721D+04    |proj g|=  1.06405D+00

At iterate  300    f=  2.03706D+04    |proj g|=  4.17668D+01

At iterate 2000    f=  2.02939D+04    |proj g|=  1.49481D+00

At iterate   50    f=  2.97676D+04    |proj g|=  2.56310D+03

At iterate 1950    f=  2.03209D+04    |proj g|=  2.21508D+00

At iterate 2000    f=  2.03239D+04    |proj g|=  1.46411D+00

At iterate 2050    f=  2.02463D+04    |proj g|=  5.35193D-01

At iterate 2000    f=  2.03398D+04    |proj g|=  1.90226D+00

At iterate 2000    f=  2.02720D+04    |proj g|=  1.92140D+00

At iterate  350    f=  2.03370D+04    |proj g|=  2.10126D+01

At iterate 2050    f=  2.02939D+04    |proj g|=  1.12505D+00

At iterate  100    f=  2.32992D+04    |proj g|=  4.03493D+02

At iterate 2000    f=  2.03209D+04    |proj g|=  4.93690D-01

At iter

In [ ]:
##########################################################################
# 5. APPROCHE DUALE : MODEL HOMME + MODEL FEMME
# + Debiasing with orthogonal projection + Optimised Logisitic Regression
##########################################################################

# ----------------------------------------------------
# PREPARATION DES DONNEES
#-----------------------------------------------------

X_F , X_H = X[S==1] , X[S==0]
Y_F , Y_H = Y[S==1] , Y[S==0]
S_F , S_H = S[S==1] , S[S==0]
print(X_F.shape,X_H.shape)
print(Y_F.shape,Y_H.shape)
print(S_F.shape,S_H.shape)


# Prepare Logistic Regression models
model_X = LogisticRegression(random_state=0, max_iter=5000,penalty='l2', C=0.2, tol=0.0001).fit(X,Y)
model_F = LogisticRegression(random_state=0, max_iter=5000,penalty='l2', C=0.2, tol=0.0001)
model_H = LogisticRegression(random_state=0, max_iter=5000, penalty='l2', C=0.2, tol=0.0001)

'''# Keep the original index for later use
X['original_index'] = X.index

# Step 1: Keep original index by resetting it if not already in a suitable format
X.reset_index(drop=True, inplace=True)
Y = Y.reset_index(drop=True)
S = S.reset_index(drop=True)

# Create a combined Y and S DataFrame for stratification
YS = pd.DataFrame({'Y': Y, 'S': S})'''

# Split the data while stratifying based on Y and S
X_train, X_test, Y_train, Y_test, S_train, S_test = train_test_split(X, Y, S, test_size=0.3, random_state=0, stratify=YS)
print('train',X_train.shape,Y_train.shape,S_train.shape)
print('test',X_test.shape,Y_test.shape,S_test.shape)

# Create XS and XnotS subsets for training and test sets
XS_train = X_train[S == True]
YS_train = Y_train[S == True]

XnotS_train = X_train[S == False]
YnotS_train = Y_train[S == False]

XS_test = X_test[S == True]
YS_test = Y_test[S == True]

XnotS_test = X_test[S == False ]
XnotS_test = X_test[S == False ]

XS_train.head(5)


print(Y_train[S == True].shape)
Y_train[S == True].head()



# Train the models
model_X.fit(X_train, Y_train)
model_XS.fit(XS_train, Y_train[S==1])
model_XnotS.fit(XnotS_train, Y_train[S==0])




**MODEL 6 - ADPAPTING LOSS FUNCTION IN LOGISTIC REGRESSION (PYTORCH)**
---
1. Functions for Custom loss function (re-written in pytorch)
2. Functions for Stratified K-fold and training batches
3. Running Model 6
4. Optimising Model 6 hyperparameters

In [ ]:
##########################################################################
# 6. CUSTOM LOSS FUNCTION IN PYTORCH (CLASSIFIER ON 28 ORIGINAL CLASSES)
# + Debiasing with orthogonal projection + Optimised Logisitic Regression
##########################################################################

**"SENSITIVE FRONTIER" MODELS <br>(USED ON DEBIASED DATASET)**
---
K nearest neighboors<br>
SVM linear<br>
SVM gaussian kernel<br>
One versus all

In [ ]:
#####################################################
#   SCORE A BLANC
#####################################################


n=Y.shape[0]
Y_pred =np.ones(n)*1
accuracy= accuracy_score(Y, Y_pred)  # Y_test are your original test labels
print(f"Accuracy on transformed test data: {accuracy}")
eval_scores, confusion_matrices_eval = gap_eval_scores(Y_pred, Y, S, metrics=['TPR'])
final_score = (eval_scores['macro_fscore']+ (1-eval_scores['TPR_GAP']))/2

#print results
print('final score',final_score)
print('macro_fscore',eval_scores['macro_fscore'])
print('1-eval_scores[\'TPR_GAP\']',1-eval_scores['TPR_GAP'])

In [ ]:
# test score "prediction uniforme"

n=Y.shape[0]

Scores_U=pd.DataFrame(columns=['N','N_f','N_h','accuracy','final_score','macro_f1','macro_gap'])

print(Scores_U)
for i in range(28):
    #Test value for all i values
    test=pd.DataFrame(np.ones(11893,dtype=int)*i)
    test.to_csv("all/Data_Challenge_all_"+str(i)+".csv", header = None, index = None)
    
    Y_pred=pd.DataFrame(np.ones(n,dtype=int)*i)
    accuracy= accuracy_score(Y, Y_pred)  # Y_test are your original test labels
    eval_scores, confusion_matrices_eval = gap_eval_scores(Y_pred, Y, S, metrics=['TPR'])
    macro_f1 = eval_scores['macro_fscore']
    macro_gap = eval_scores['TPR_GAP']
    final_score = (macro_f1 +1- macro_gap)/2
    # check number of occurence
    N = (Y==i).sum()
    N_f = (Y56 == i + 28).sum()
    N_h = (Y56 == i).sum()
    Scores_U.loc[i]= [N, N_f, N_h, accuracy,final_score,macro_f1,macro_gap]

Scores_U

In [ ]:
size = Scores_U['N']
score = Scores_U['final_score']


# Créer et entraîner le modèle de régression linéaire
from sklearn.linear_model import LinearRegression
reverse = LinearRegression()
reverse.fit(np.array(score).reshape(-1, 1),np.array(size).reshape(-1, 1))

# Load the scores for X_true
scores_true = pd. read_csv('all/distribution_Y_true.txt',header = None,index_col=0)
dist_true_pred = reverse.predict(scores_true) #/ 11893 * 27749

# Afficher la distribution de X_test_true
plt.scatter(size,score)
plt.scatter(dist_true_pred,scores_true)
plt.show

In [ ]:
plt.scatter(Scores_U['final_score'],scores_true)

In [ ]:
plt.plot(range(28),size,label='X')
plt.plot(range(28),np.round(dist_true_pred),'X_test_true')
plt.legend()

In [ ]:
# Normalize data (L2 norm recommended for embeddings)
#X = normalize(X, norm='l2')
#X_test_true = normalize(X_test_true, norm='l2')



In [ ]:
##############################################################
# train_test_split with Y56 (np.arrays)
##############################################################

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, Y56_train, Y56_test = train_test_split(X, Y56, test_size=0.2, random_state=42)
Y_train = Y56_train % 28  # reste (original Y)   ex 33% 28 = classe 5 
S_train = Y56_train//28   # facteur (original S) ex 33//28 = 1 (attribut protégé)
Y_test = Y56_test % 28  # reste (original Y)   ex 33% 28 = classe 5 
S_test = Y56_test//28   # facteur (original S) ex 33//28 = 1 (attribut protégé)

# impression des dimensions
print('train:',X_train.shape,Y_train.shape,S_train.shape)
print('test:',X_test.shape,Y_test.shape, S_test.shape)

**Explore data**
---

In [ ]:
dist = pd.DataFrame({'label':Y,'S':S})

# Sum counts of 1s and 0s to get the total count for each label
result = dist.groupby('label')['S'].value_counts().unstack(fill_value=0)
result.columns = ['S', 'not_S']
result['total_count'] = result.sum(axis=1)

# Calculate totals for each column
totals = result.sum(axis=0)
result.loc[30] = totals

# Calculate total count percentages for each count
result['%_S_label'] = round((result['S'] / result['total_count']) * 100)
result['%_not_S_label'] = round((result['not_S'] / result['total_count']) * 100)
result['%_total'] = np.round(result['total_count']/len(Y)*100,2 ) # % of total count
result['%_S_total'] = np.round(result['S']/(S==1).sum()*100,2 ) # % of total count
result['%_not_S_total'] = np.round(result['not_S']/(S!=1).sum()*100,2 ) # % of total count
result['diff_%_S_total']=result['%_S_total']-result['%_total']
result['|']='|'
#Reorder table
result = result.reindex(columns=['total_count', '%_total','|','S','%_S_total','diff_%_S_total','|','not_S', '%_not_S_total','|','%_S_label', '%_not_S_label'])


result

In [ ]:
# Extract the data from the '%_S' column
labels = pd.to_numeric(result.index)
data = result['%_total']
data_S = result['%_S_total']
data_not_S = result['%_not_S_total']

# Create a bar plot
plt.plot(labels[:-1], data[:-1])
plt.plot(labels[:-1],data_S[:-1], label='Sensitive')
plt.plot(labels[:-1],data_not_S[:-1], label='Not_sensitive')

# Add labels and title
plt.xlabel('Label')
plt.ylabel('Percentage of total')
plt.title('Percentage of total for sample, S and non-S')
plt.legend()
# Show the plot
plt.show()

In [ ]:
result_sorted = result.iloc[:-1].sort_values(by='diff_%_S_total', ascending=False)
result_sorted['original_label'] = result_sorted.index
result_sorted=result_sorted.reset_index(drop=True)
#result_sorted.reset_index(drop=True, inplace=True)
result_sorted

In [ ]:
# Extract the data from the '%_S' column
labels_sorted = result_sorted['original_label']
#print(labels_sorted)
data = result_sorted['%_total']
data_S = result_sorted['%_S_total']
data_not_S = result_sorted['%_not_S_total']

# Create a bar plot
plt.plot(data)
plt.plot(data_S, label='Sensitive')
plt.plot(data_not_S, label='Not_sensitive')

# Add labels and title
plt.xlabel('Label')
plt.ylabel('Percentage of total')
#plt.xticks(labels_sorted)
plt.xticks(ticks=range(len(labels_sorted)), labels=labels_sorted, rotation=90)  # Rotate if there are many labels

plt.title('Percentage of total for sample, S and non-S')
plt.legend()
# Show the plot
plt.show()

**MODEL ORTHOGONAL DEBIASING**
---

In [ ]:
import torch


# Fonction de similarité cosinus
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

# Pré-générer les embeddings pour chaque token dans le vocabulaire
# ATTENTION : Cette étape est très coûteuse !
vocab_embeddings = {}
for word, token_id in tokenizer.vocab.items():
    input_ids = torch.tensor([tokenizer.convert_tokens_to_ids(word)]).unsqueeze(0)  # Ajouter batch dimension
    with torch.no_grad():
        outputs = model(input_ids)
    vocab_embeddings[word] = outputs.last_hidden_state[0, 0, :].numpy()  # Utiliser l'embedding du token

# Supposons que X soit votre matrice d'embeddings (n, 768)
# X = ...

# Trouver le mot le plus proche pour les 10 premiers embeddings
for i in range(10):
    embedding = X[i]
    similarities = {word: cosine_similarity(embedding, word_emb) for word, word_emb in vocab_embeddings.items()}
    closest_word = max(similarities, key=similarities.get)
    print(f"Ligne {i}: Mot le plus proche = {closest_word}")


**baseline Logistic regression on raw data**
---

In [ ]:
print("Coefficients:", clf_1.coef_.shape)
print("Intercept:", clf_1.intercept_.shape)

In [ ]:
print(eval_scores_1)
print(confusion_matrices_eval_1.keys)

#show confusion matrix for Accurarcy key 0
pd.DataFrame(confusion_matrices_eval_1[0])

In [ ]:
# (Re)Load the "true" test data
X_test_true = dat['X_test']
S_test_true = dat['S_test'] 

# Classify the provided test data with you classifier
y_test_true_1 = clf_1.predict(X_test_true)
results_1=pd.DataFrame(y_test_true_1, columns= ['score'])

results_1.to_csv("Data_Challenge_MDI_341_1.csv", header = None, index = None)
# np.savetxt('y_test_challenge_student.txt', y_test, delimiter=',')

**SECOND METHOD - ADVERSARIAL NN**
---

In [ ]:
# Refresh training data
# X_train, X_test, Y_train, Y_test, S_train, S_test = X_train_, X_test_, Y_train_, Y_test_, S_train_, S_test_

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy , CategoricalCrossentropy

from tensorflow.keras.metrics import AUC
from tensorflow.keras.utils import to_categorical

# Assuming X_train is your input embeddings and S is your sensitive attribute

# Input layer
input_layer = Input(shape=(768,))

# Main task classifier layers
main_task_hidden = Dense(256, activation='relu')(input_layer)
main_task_output = Dropout(0.5)(main_task_hidden)
main_task_output = Dense(28, activation='softmax', name='main_task_output')(main_task_hidden)

# Adversarial component layers
adversary_hidden = Dense(256, activation='relu')(main_task_hidden)
adversary_hidden = Dropout(0.5)(adversary_hidden)
adversarial_output = Dense(1, activation='sigmoid', name='adversarial_output')(adversary_hidden)

# Model
model_2 = Model(inputs=input_layer, outputs=[main_task_output, adversarial_output])

# Optimizers
#main_task_optimizer = Adam(learning_rate=0.001)
#adversarial_optimizer = Adam(learning_rate=0.001)

# Loss functions
main_task_loss = CategoricalCrossentropy()
adversarial_loss = BinaryCrossentropy()

# Compile the model
model_2.compile(optimizer=Adam(learning_rate=0.001),
              loss={'main_task_output': main_task_loss, 'adversarial_output': adversarial_loss},
              loss_weights=[1, -0.1],
              metrics={'main_task_output': ['accuracy'], 'adversarial_output': [AUC()]})

# Prepare the labels for the main task and the adversarial task
Y_main_task = to_categorical(Y_train, num_classes=28)#Y_train # Your main task labels
Y_adversary = S_train    # Your sensitive attribute labels

# check size on input .output
print(X_train.shape,Y_main_task.shape,Y_adversary.shape)

# X normal:isation
# none in method 2

# Train the model
model_2.fit(X_train, {'main_task_output': Y_main_task, 'adversarial_output': Y_adversary}, epochs=10)

# After training, you can use the output of `main_task_hidden` as your new unbiased representation


In [ ]:
# Step 1: Transform X_train and X_test
feature_extractor_2 = Model(inputs=model_2.input, outputs=main_task_hidden)
X_train_transformed_2 = feature_extractor_2.predict(X_train)
X_test_transformed_2 = feature_extractor_2.predict(X_test)
X_test_true_transformed_2 = feature_extractor_2.predict(X_test_true)

# Step 2: Train a new classifier on the transformed training data
clf_2 = LogisticRegression(max_iter=5000)  # Increase max_iter if needed for convergence
history_new_2 = clf_2.fit(X_train_transformed_2, Y_train)  # Y_train are your original training labels

# Step 3: Predict on the transformed test data and evaluate
Y_pred_2 = clf_2.predict(X_test_transformed_2)
accuracy_2= accuracy_score(Y_test, Y_pred_2)  # Y_test are your original test labels
print(f"Accuracy on transformed test data: {accuracy_2}")

# Step 4 : Predict with gloabl score
eval_scores_2, confusion_matrices_eval_2 = gap_eval_scores(Y_pred_2, Y_test, S_test, metrics=['TPR'])
final_score_2 = (eval_scores_2['macro_fscore']+ (1-eval_scores_2['TPR_GAP']))/2
print('\nfinal',final_score_2)
print('macro_fscore',eval_scores_2['macro_fscore'])
print('1-eval_scores',1-eval_scores_2['TPR_GAP'])

In [ ]:
# Classify the provided test data with you classifier
y_test = clf_2.predict(X_test_true_transformed_2)
results=pd.DataFrame(y_test, columns= ['score'])

results.to_csv("Data_Challenge_MDI_341_2.csv", header = None, index = None)
# np.savetxt('y_test_challenge_student.txt', y_test, delimiter=',')

**THIRD METHOD**
---

In [ ]:
from sklearn.preprocessing import StandardScaler

# get X_train an
# X_train, X_test, Y_train, Y_test, S_train, S_test = train_test_split(X, Y, S, test_size=0.3, random_state=42)

# Initialize StandardScaler
scaler = StandardScaler()

# Fit the scaler to your training data
scaler.fit(X_train)

# Transform your training data
X_train_standardized = scaler.transform(X_train)
X_test_standardized = scaler.transform(X_test)
X_test_true_standardized = scaler.transform(X_test_true)



In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy , CategoricalCrossentropy

from tensorflow.keras.metrics import AUC
from tensorflow.keras.utils import to_categorical

# Assuming X_train is your input embeddings and S is your sensitive attribute

# Input layer
input_layer = Input(shape=(768,))

# Main task classifier layers
main_task_hidden = Dense(256, activation='relu')(input_layer)
main_task_hidden = Dropout(0.5)(main_task_hidden)
main_task_output = Dense(28, activation='softmax', name='main_task_output')(main_task_hidden)

# Adversarial component layers
adversary_hidden = Dense(256, activation='relu')(main_task_hidden)
adversary_hidden = Dropout(0.5)(adversary_hidden)
adversarial_output = Dense(1, activation='sigmoid', name='adversarial_output')(adversary_hidden)

# Model
model_3 = Model(inputs=input_layer, outputs=[main_task_output, adversarial_output])

# Optimizers
#main_task_optimizer = Adam(learning_rate=0.001)
#adversarial_optimizer = Adam(learning_rate=0.001)

# Loss functions
main_task_loss = CategoricalCrossentropy()
adversarial_loss = BinaryCrossentropy()

# Compile the model
model_3.compile(optimizer=Adam(learning_rate=0.001),
              loss={'main_task_output': main_task_loss, 'adversarial_output': adversarial_loss},
              metrics={'main_task_output': ['accuracy'], 'adversarial_output': [AUC()]})

# Prepare the labels for the main task and the adversarial task
Y_main_task = to_categorical(Y_train, num_classes=28)#Y_train # Your main task labels
Y_adversary = S_train    # Your sensitive attribute labels

# check size on input .output
print(X_train_standardized.shape,Y_main_task.shape,Y_adversary.shape)

# Train the model
model_3.fit(X_train_standardized, {'main_task_output': Y_main_task, 'adversarial_output': Y_adversary}, epochs=10)

# After training, you can use the output of `main_task_hidden` as your new unbiased representation


In [ ]:
# Step 1: Transform X_train and X_test
feature_extractor_3 = Model(inputs=model_3.input, outputs=main_task_hidden)
X_train_transformed_3 = feature_extractor_3.predict(X_train_standardized)
X_test_transformed_3 = feature_extractor_3.predict(X_test_standardized)
X_test_true_transformed_3 = feature_extractor_3.predict(X_test_true_standardized)

# Step 2: Train a new classifier on the transformed training data
new_classifier_3 = LogisticRegression(max_iter=10000)  # Increase max_iter if needed for convergence
history_new_3 = new_classifier_3.fit(X_train_transformed_3, Y_train)  # Y_train are your original training labels

# Step 3: Predict on the transformed test data and evaluate
Y_pred_3 = new_classifier_3.predict(X_test_transformed_3)
accuracy_3= accuracy_score(Y_test, Y_pred_3)  # Y_test are your original test labels
print(f"Accuracy on transformed test data: {accuracy_3}")

In [ ]:
# calculate final score
eval_scores, confusion_matrices_eval = gap_eval_scores(Y_pred_3, Y_test, S_test, metrics=['TPR'])
final_score = (eval_scores['macro_fscore']+ (1-eval_scores['TPR_GAP']))/2

print('macro_fscore',eval_scores['macro_fscore'])
print('1-eval_scores',1-eval_scores['TPR_GAP'])
print('final score (average)',final_score)

In [ ]:
# Load the "true" test data
X_test_true = dat['X_test']
S_test_true = dat['S_test'] 

X_test_true_transformed_3 = feature_extractor_3.predict(X_test_true)

# Classify the provided test data with you classifier
y_test_true = clf.predict(X_test_true_transformed_3)
results_3=pd.DataFrame(y_test_true, columns= ['score'])

results_3.to_csv("Data_Challenge_MDI_341_3.csv", header = None, index = None)
# np.savetxt('y_test_challenge_student.txt', y_test, delimiter=',')

**4th METHOD**
---

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy , CategoricalCrossentropy

from tensorflow.keras.metrics import AUC
from tensorflow.keras.utils import to_categorical

# Assuming X_train is your input embeddings and S is your sensitive attribute

# Input layer
input_layer = Input(shape=(768,))

# Main task classifier layers
main_task_hidden = Dense(512, activation='relu')(input_layer)
main_task_hidden = Dropout(0.5)(main_task_hidden)
main_task_output = Dense(28, activation='softmax', name='main_task_output')(main_task_hidden)

# Adversarial component layers
adversary_hidden = Dense(512, activation='relu')(main_task_hidden)
adversary_hidden = Dropout(0.5)(adversary_hidden)
adversarial_output = Dense(1, activation='sigmoid', name='adversarial_output')(adversary_hidden)

# Model
model_4 = Model(inputs=input_layer, outputs=[main_task_output, adversarial_output])

# Optimizers
#main_task_optimizer = Adam(learning_rate=0.001)
#adversarial_optimizer = Adam(learning_rate=0.001)

# Loss functions
main_task_loss = CategoricalCrossentropy()
adversarial_loss = BinaryCrossentropy()

# Compile the model
model_4.compile(optimizer=Adam(learning_rate=0.001),
              loss={'main_task_output': main_task_loss, 'adversarial_output': adversarial_loss},
              metrics={'main_task_output': ['accuracy'], 'adversarial_output': [AUC()]})

# Prepare the labels for the main task and the adversarial task
Y_main_task = to_categorical(Y_train, num_classes=28)#Y_train # Your main task labels
Y_adversary = S_train    # Your sensitive attribute labels

# check size on input .output
print(X_train_standardized.shape,Y_main_task.shape,Y_adversary.shape)

# Train the model
model_4.fit(X_train_standardized, {'main_task_output': Y_main_task, 'adversarial_output': Y_adversary}, epochs=10)

# After training, you can use the output of `main_task_hidden` as your new unbiased representation


In [ ]:
# Step 1: Transform X_train and X_test
feature_extractor_4 = Model(inputs=model_4.input, outputs=main_task_hidden)
X_train_transformed_4 = feature_extractor_4.predict(X_train_standardized)
X_test_transformed_4 = feature_extractor_4.predict(X_test_standardized)
X_test_true_transformed_4 = feature_extractor_4.predict(X_test_true_standardized)

# Step 2: Train a new classifier on the transformed training data
new_classifier_4 = LogisticRegression(max_iter=10000)  # Increase max_iter if needed for convergence
history_4 = new_classifier_4.fit(X_train_transformed_4, Y_train)  # Y_train are your original training labels

# Step 3: Predict on the transformed test data and evaluate
Y_pred_4 = new_classifier_4.predict(X_test_transformed_4)
accuracy_4 = accuracy_score(Y_test, Y_pred_4)  # Y_test are your original test labels
print(f"Accuracy on transformed test data: {accuracy_4}")

In [ ]:
eval_scores, confusion_matrices_eval = gap_eval_scores(Y_pred_4, Y_test, S_test, metrics=['TPR'])
#eval_scores#eval_scores['macro_fscore']
#eval_scores['TPR_GAP']
final_score = (eval_scores['macro_fscore']+ (1-eval_scores['TPR_GAP']))/2
print(final_score)

In [ ]:
# Load the "true" test data
X_test_true = dat['X_test']
S_test_true = dat['S_test'] 

X_test_true_transformed_4 = feature_extractor_4.predict(X_test_true)

# Classify the provided test data with you classifier
y_test_true = clf.predict(X_test_true_transformed_4)
results_4=pd.DataFrame(y_test_true, columns= ['score'])

results_4.to_csv("Data_Challenge_MDI_341_4.csv", header = None, index = None)
# np.savetxt('y_test_challenge_student.txt', y_test, delimiter=',')